In [3]:
#import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, StratifiedKFold

from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.dictionary_based import MUSE

In [2]:
# get working directory and remove last folder
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  /Users/nzuchna/Desktop/Drive/2. Areas/University/Master-TUC/M2/4_Forschungsmodul/student-performance


df_0_4 = pd.read_csv('data/processed/df_0_4_flattened.csv')
labels = pd.read_csv('data/processed/labels.csv')

df_0_4.head(10)

df_edit = df_0_4.drop(columns=['Unnamed: 0', 'level_group_1'])
df_edit.head()

#df_edit = df_edit[~df_edit['session_id_1'].isin(labels.session)]
#labels = labels[~labels['session'].isin(df_edit.session_id_1)]

labels_q1_3 = labels[labels['q'] <= 3]

print(df_edit.shape, labels_q1_3.shape)

X1 = df_edit.drop(columns=['session_id_1'])
y1 = labels.drop(columns=['session', 'q'])

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, train_size=.75) # , test_size=0.5
print(X_train1.shape, y_train1.shape, X_test1.shape, y_test1.shape)

In [4]:
# X
df = pd.read_csv("data/processed/df_5_12_flattened.csv", index_col=0)
X = df.iloc[:, 2:].values
# X = np.delete(X, 61, 1)

# y
labels = pd.read_csv("data/processed/labels_q4-13.csv")
y = labels['correct'][labels['q'] == 10].values

# Creating training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify = y)

# Feature Scaling

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

def eval_model(model, cm_title: str, ):
    # Model evalutation
    y_pred = model.predict(X_test)
    # print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

    # fmt='d' formats the numbers as digits, which means integers
    cm = confusion_matrix(y_test, y_pred)
    hp = sns.heatmap(cm, annot=True, fmt='d')
    hp.set_title(f"Confusion matrix of {cm_title}")
    hp.set(xlabel='Predicted', ylabel='Actual')

    print(classification_report(y_test,y_pred))

In [5]:
%%time
rocket1 = RocketClassifier()

rocket1.fit(X_train, y_train)

rocket_preds = rocket1.predict(X_test)
print("Rocket1 Accuracy: " + str(metrics.accuracy_score(y_test, rocket_preds)))

Rocket1 Accuracy: 0.5054463148960249
CPU times: user 55min 49s, sys: 1min 23s, total: 57min 12s
Wall time: 29min 14s


In [ ]:
eval_model(rocket1, 'Rocket1')

In [6]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(16493, 170) (16493,) (7069, 170) (7069,)
